# This notebook includes all the steps for going from the RS files to the files you will need for training and testing your model 

In [1]:
import os
import sqlite3
import json
from datetime import datetime
import pandas as pd
import pickle
import glob
from carsonNLP.embedding import Vocabulary

# Dataset

## How to allocate the RS_YEAR-DAY files, and what are the intermediate files

Put all the RS files into one folder 



In [2]:
path_to_RS = "RS"
all_filepaths = glob.glob(path_to_RS+'/RS*')    

## The old class to subreddit mapping

In [ ]:
label2subreddit_dict =  {
'conversation' = ['CasualConversation', 'relationships','AskReddit','Jokes', 'CoolStoryBro', 
                'stupidpeople','Showerthoughts','DoesAnybodyElse','changemyview','Foodforthought',
                'announcements','funny','AskReddit','todayilearned','IAmA','news'],
'happy' = ['happy','UpliftingNews', 'feelgood', 'goodfeelings', 'ThankYou', 'GetMotivated', 
         'DecidingToBeBetter', 'benicetopeople' ,'AccomplishedToday','InspirationalMoments'],
'anxiety' = ['Anxiety','socialanxiety','AnxietyPanic'],
'addiction' = ['addiction','opiates','OpiatesRecovery','cripplingalcoholism'],
'depression' = ['depression', 'StopSelfHarm', 'selfharm', 'SuicideWatch'],
'bipolar' = ['BPD','bipolar','BipolarSOs'],
'autism' = ['autism','aspergers'],
'schizophrenia' = ['schizophrenia']
}

## choose unacceptable strings
do not use submission/posts that have these unacceptable strings in their 'selftext' field
## choose a minimum and maximum string length for inclusion
this is the whole string including all characters and spaces, make this range the range that you think meaningful posts come in, not the range that will fit into a minibatch

In [5]:
unacceptable = ['[removed]','[deleted]']

min_chars = 10
max_chars = 10000

'''   
label2subreddit_dict = {

'conversation' : ['CasualConversation', 'relationships','AskReddit', 'CoolStoryBro', 
                'stupidpeople','Showerthoughts','DoesAnybodyElse','changemyview','Foodforthought',
                'announcements','AskReddit','todayilearned','IAmA','news','gossip'],

'jokes': ['jokes','funny'],
    
'happy' : ['happy','UpliftingNews', 'feelgood', 'goodfeelings', 'ThankYou', 
           'AccomplishedToday','InspirationalMoments',
           'faithinhumanity','gratitude','goodnews','MadeMeSmile','moodboost',
           'optimism','palatecleanser','positivity','thankyou','wherepeoplecare',
            'DecidingToBeBetter','small_act_of_kindness_from_a_friend', 'randomactsofkindness'],

'anxiety' : ['Anxiety','socialanxiety','AnxietyPanic'],

'addiction' : ['addiction','opiates','OpiatesRecovery','cripplingalcoholism'],

'depression' : ['depression'], 
    
 'selfharm'  : ['StopSelfHarm', 'selfharm', 'SuicideWatch'],

'bipolar' : ['BPD','bipolar','BipolarSOs'],

'autism' : ['autism','aspergers'],

'schizophrenia' : ['schizophrenia']

}
'''

label2subreddit_dict =  {
'conversation' : ['CasualConversation', 'relationships','AskReddit','Jokes', 'CoolStoryBro', 
                'stupidpeople','Showerthoughts','DoesAnybodyElse','changemyview','Foodforthought',
                'announcements','funny','AskReddit','todayilearned','IAmA','news'],
'happy' : ['happy','UpliftingNews', 'feelgood', 'goodfeelings', 'ThankYou', 'GetMotivated', 
         'DecidingToBeBetter', 'benicetopeople' ,'AccomplishedToday','InspirationalMoments'],
'anxiety' : ['Anxiety','socialanxiety','AnxietyPanic'],
'addiction' : ['addiction','opiates','OpiatesRecovery','cripplingalcoholism'],
'depression' : ['depression', 'StopSelfHarm', 'selfharm', 'SuicideWatch'],
'bipolar' : ['BPD','bipolar','BipolarSOs'],
'autism' : ['autism','aspergers'],
'schizophrenia' : ['schizophrenia']
}

subreddit2label_dict = {}
for key, value in label2subreddit_dict.items():
    for subreddit in value:
        subreddit2label_dict[subreddit.lower()]=key.lower()

data_count = dict.fromkeys(label2subreddit_dict, 0)

subreddit_list = []
for key, value in label2subreddit_dict.items():
    subreddit_list.extend([v.lower() for v in value])
    
happy_subreddit_dict = {}
for happy_subreddit in label2subreddit_dict['happy']:
    happy_subreddit_dict[happy_subreddit.lower()] = []

default - rumination 

salience - axious avoidance

negative affect - negative bias, threat dysregulation

positive affect   - anhedonia (reward hypoactivation), context insensitivity

attention - inatttention

congnitive control - cognitive dyscontrol

In [6]:
class_folder_path = 'class_folder_path3'

if not os.path.exists(class_folder_path):
    os.mkdir(class_folder_path)
    
for label in label2subreddit_dict.keys():
    label_doc_path = os.path.join(class_folder_path,label+'.txt')
    if not os.path.exists(label_doc_path):
        file = open(label_doc_path,'w')
        file.close()

### The raw text is placed, the function normalize_string not used until the embedding and data_dict stages

### I made the decision to use add the title to the beginning of the post and use only the title when the post is not in range 

In [7]:
# this for loop takes more 1 - 2 hours to run on 8 RS files
for filepaths in all_filepaths:
    print(filepaths)
    with open(filepaths, buffering=100) as f:
        for row in f:
            row = json.loads(row)
            if row['selftext'] not in unacceptable \
            and 'subreddit' in row.keys() \
            and row['subreddit'].lower() in subreddit_list:
                label = subreddit2label_dict[row['subreddit'].lower()]
                label_doc_path = os.path.join(class_folder_path,label+'.txt')
                file = open(label_doc_path,'a')
                if len(row['selftext'].strip()) in range(min_chars,max_chars):
                    file.write(row['title'] + ' , ' + row['selftext']+"\n") 
                else:
                    file.write(row['title']+"\n")
                file.close()
                
                if label == 'happy':
                    happy_subreddit_dict[row['subreddit'].lower()].append(row['title'] + ' , ' + row['selftext'])

RS/RS_2017-06
RS/RS_2017-07
RS/RS_2017-08
RS/RS_2017-09
RS/RS_2017-10
RS/RS_2017-11
RS/RS_2017-12
RS/RS_2018-01
RS/RS_2018-02


In [13]:
max_len = 1000
min_len = 10
path_to_folder_of_text = 'class_folder_path2'
all_filenames = glob.glob(path_to_folder_of_text+'/*.txt')

vocabClass = Vocabulary()

training_dict = {}
validation_dict = {}
test_dict = {}
all_data = {}
all_categories = []

# training validation split here, the val and test set are balanced because the same number of samples are taken
# from the end of each document class to be placed into this set, but the training set might still be unbalanced
num_test = 1000 # number of test samples per class
num_validation = 100 # number of validation samples per class
last_train = -num_test - num_validation
print("index of first validation sample",last_train)
last_validation = -num_test
print("index of first test sample",last_validation)

for filename in all_filenames:
    
    category = filename.split('/')[-1].split('.')[0] # get label name from file
    all_categories.append(category) # add labe to list of labels
    lines = open(filename).read().strip().split('\n') # split based on the newline delimiter
    lines = [vocabClass.normalize_string(line) for line in lines if len(line) in range(min_len,max_len)] 
    print(category,len(lines))
    all_data[category] = lines
    training_dict[category] = lines[:last_train]
    validation_dict[category] = lines[last_train:last_validation]
    test_dict[category] = lines[last_validation:]

index of first validation sample -1100
index of first test sample -1000
addiction 51116
anxiety 58900
autism 19887
bipolar 43710
conversation 2047037
depression 130572
happy 47216
jokes 466398
schizophrenia 10148
selfharm 53799


In [14]:
emotrvaltest = (training_dict, validation_dict, test_dict, all_data, all_categories)
pickle.dump(emotrvaltest, open("trvaltest/emotrvaltest_human.p", "wb" ) )

# Vocabulary

our initial vocabulary had  18024 tokens, we only used lowercase words, tha can be changed easily by changing

s = self.unicode_to_ascii(s.lower()) to s = self.unicode_to_ascii(s)

only a-zA-Z0-9,.!? are kept 

In [11]:
import torchtext.vocab as vocab
glove = vocab.GloVe(name='6B', dim=100) # available in 100
print('Loaded {} words'.format(len(glove.itos)))

Loaded 400000 words


In [12]:
vocabClass = Vocabulary()
min_word_count = 30
path_to_folder_of_text = 'class_folder_path2'
index2word, word2index, embedding = vocabClass.makeEmbedding(min_word_count, glove, path_to_folder_of_text)
dicts_embed = (index2word, word2index, embedding)
pickle.dump(dicts_embed, open("embeddings/dicts_embed_min30.p", "wb" ) )

trimming...
keep_words 30613 / 324298 = 0.0944
building embedding from glove...
finished embeddings


In [16]:
st = vocabClass.normalize_string("      This    don't,?! that's you're \"no\" ")
print(st)
"\" no \"" in st

this dont , ? ! thats youre no


False